# import

In [1]:
import os
import numpy as np
import math
import pandas as pd
import re
import shutil
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# make gjf

In [2]:
class make_gjf():
    def __init__(self):
        self.theta_list=[40.0,50.0,60.0]
        self.R3_list=[round(R3,1) for R3 in np.linspace(-4.0,4.0,81)]
        self.path='C:\\Users\\youmo\\OSC\\interaction\\BTBT\\'
        self.dimer_path='C:\\Users\\youmo\\OSC\\interaction\\BTBT\\dimer_Z_opt.gjf'
        
    def make_arr(self,R1,R2,R3,theta_,torp):
        with open(self.dimer_path,'r') as f:
                xyz=f.readlines()
        xyz[4]=torp+'\n'#配置
        xyz[66]='XX '+str(30+R1)+'\n'
        xyz[67]='YY '+str(30-R2)+'\n'
        xyz[68]='ZZ '+str(10-R3)+'\n'
        xyz[69]='BB '+str(theta_)+'\n'
        xyz_add=xyz+['--Link1--\n']
        return xyz_add

    def convertor_R(self,Ra,Rb,theta_):
        R1=Ra*np.cos(theta_)+Rb*np.sin(theta_)
        R2=-Ra*np.sin(theta_)+Rb*np.cos(theta_)
        return R1,R2
    
    #t字分子(Z=2)を向こうから見て基準分子のように見立てる
    #Z=2ではRa=Ra,Rb=-Rbとする
    
##### make inp #####
    def make_step1(self,in_csv,glide_mode,edge_mode):#参考にするvdwのcsv、a,b映進どちらか、a,b端どちらか
        df_contact=pd.read_csv('C:\\Users\\youmo\\OSC\\interaction\\BTBT\\vdw\\'+in_csv)
        
        for A3 in [26,27,28,29,31,32,33,34]:#ここもカスタマイズ
            A1,A2,A3_,theta_ab,R,S,a,b=df_contact[df_contact['A3']==int(A3)].values[0]#レコードが1つしかないので0入れる
            A3_rad=np.radians(A3)
            R1_center=(a*np.cos(A3_rad)+b*np.sin(A3_rad))/2
            R2_center=(b*np.cos(A3_rad)-a*np.sin((A3_rad)))/2
            
            #R1,R2の範囲をカスタマイズ
            self.R1_list=[np.round(d1+R1_center,1) for d1 in np.linspace(-0.5,0.5,11)]
            self.R2_list=[np.round(d2+R2_center,1) for d2 in np.linspace(-0.5,0.5,11)]
            
            for R1 in self.R1_list:
                for R2 in self.R2_list:
                    b_=2*(R1*np.sin(A3_rad)+R2*np.cos(A3_rad))
                    a_=2*(R1*np.cos(A3_rad)-R2*np.sin(A3_rad))
                    
                    #edgeによって平行配置a,bのどちらが近くなるかが変わる。
                    if edge_mode=='a':#普通こっち
                        R1_p=b_*np.sin(A3_rad);R2_p=b_*np.cos(A3_rad)
                    elif edge_mode=='b':
                        R1_p=a_*np.sin(A3_rad);R2_p=-a_*np.cos(A3_rad) #修正した4/24 edge=bはやり直し。。。
                        
                    #映進面によってT配置の分子の向きが違う
                    if glide_mode=='b':
                        xyz_t= self.make_arr(R1,R2,0.0,float(A3*2),'t')
                    elif glide_mode=='a':
                        xyz_t= self.make_arr(R1,R2,0.0,A3*2+180.0,'t')
                    
                    xyz_p= self.make_arr(R1_p,R2_p,0.0,0.0,'p')
                    xyz_list=['$ RunGauss\n']+xyz_t+xyz_p[:-1]#
                    with open(self.path+'step1\\glide='+glide_mode+'\\edge='+edge_mode+'\\R1={}_R2={}_theta={}.inp'.format(R1,R2,int(2*A3)),'w') as f:
                        f.writelines(xyz_list)

    def make_step2(self,R1,R2,theta,glide_mode,edge_mode):
        theta_rt=np.radians(float(theta))
        b_=2*(R1*np.sin(theta_rt/2)+R2*np.cos(theta_rt/2))
        a_=2*(R1*np.cos(theta_rt/2)-R2*np.sin(theta_rt/2))
        R1_pb=b_*np.sin(theta_rt/2);R2_pb=b_*np.cos(theta_rt/2)
        R1_pa=a_*np.sin(theta_rt/2);R2_pa=-a_*np.cos(theta_rt/2)
        xyz_t1_list,xyz_t2_list,xyz_p_list=['$ RunGauss\n'],['$ RunGauss\n'],['$ RunGauss\n']
        
        #T-shape-1
        theta_t=theta+180.0 if glide_mode=='a' else theta+0.0
        for R3_t1 in self.R3_list:
            xyz_t1= self.make_arr(R1,R2,R3_t1,theta_t,'t1')
            xyz_t1_list=xyz_t1_list+xyz_t1
            
        if edge_mode=='a':#a端の場合
            #parallel
            for R3_p in self.R3_list:
                xyz_p=self.make_arr(R1_pb,R2_pb,R3_p,0.0,'p')
                xyz_p_list=xyz_p_list+xyz_p
            #T-shape-2
            for R3_t2 in self.R3_list:
                xyz_t2= self.make_arr(R1-R1_pb,R2-R2_pb,R3_t2,theta_t,'t2')
                xyz_t2_list=xyz_t2_list+xyz_t2
        
        elif edge_mode=='b':#a端の場合
            for R3_p in self.R3_list:
                xyz_p=self.make_arr(R1_pa,R2_pa,R3_p,0.0,'p')
                xyz_p_list=xyz_p_list+xyz_p
            for R3_t2 in self.R3_list:
                xyz_t2= self.make_arr(R1_pa-R1,R2_pa-R2,R3_t2,theta_t,'t2')
                xyz_t2_list=xyz_t2_list+xyz_t2
        
        with open(self.path+'step2\\glide={}\\edge={}\\t1.inp'.format(glide_mode,edge_mode),'w') as f:
            f.writelines(xyz_t1_list[:-1])
        with open(self.path+'step2\\glide={}\\edge={}\\t2.inp'.format(glide_mode,edge_mode),'w') as f:
            f.writelines(xyz_t2_list[:-1])
        with open(self.path+'step2\\glide={}\\edge={}\\p.inp'.format(glide_mode,edge_mode),'w') as f:
            f.writelines(xyz_p_list[:-1])    
        
        #step1_min,step2_min,vdwmin.glideを入れる
    def make_step3(self,R1_t,R2_t,heri,R3_t,R3_p,Ra_i0,Rb_i0,R3_i0,glide_mode,edge_mode,Z):#edge='a','b',Z=1,2
        
        glide= 180.0 if glide_mode=='a' else 0.0
        #p配置のベクトルのための準備
        heri_rt=np.radians(heri)
        b_=2*(R1_t*np.sin(heri_rt/2)+R2_t*np.cos(heri_rt/2))
        a_=2*(R1_t*np.cos(heri_rt/2)-R2_t*np.sin(heri_rt/2))
        R1_p=b_*np.sin(heri_rt/2);R2_p=b_*np.cos(heri_rt/2)
        
        #t,p,i0配置の相対位置ベクトル
        R1_i0,R2_i0=self.convertor_R(Ra_i0,Rb_i0,heri_rt/2)
        R_t=np.array([R1_t,R2_t,R3_t])
        R_p=np.array([R1_p,R2_p,R3_p])
        R_i0=np.array([R1_i0,R2_i0,R3_i0])#層2のZ=1(手前・上)
        inter_7list=[[0,0,'p'],[0,1,'p'],[1,0,'t'],[1,-1,'t'],
                     [0,-1,'p'],[-1,0,'t'],[-1,1,'t']]
        xyz_list=['$ RunGauss\n']

        for ind,arr in enumerate(inter_7list):
            t=arr[0];p=arr[1]
            R=-R_i0+(R_t*t+R_p*p) #層2の基準分子(Z=1)からみた奥の層1の分子の位置ベクトル
            theta=heri+glide if arr[2]=='t' else 0.0
            xyz_= self.make_arr(R[0],R[1],R[2],theta,arr[2]+str(ind))
            xyz_list=xyz_list+xyz_

        with open(self.path+'step3\\glide='+glide_mode+'\\edge='+edge_mode+'\\R1={}_R2={}_heri={}_R3t={}_R3p={}_Ra_i0={}_Rb_i0={}_R3_i0={}_glide={}_Z={}.inp'.format(R1_t,R2_t,heri,R3_t,R3_p,Ra_i0,Rb_i0,R3_i0,glide_mode,Z),'w') as f:
            f.writelines(xyz_list[:-1])

####### チェック用 #######
    def view(self,R1_t,R2_t,heri,R3_t,R3_p,R1_i0,R2_i0,R3_i0,glide_mode):
        b_=2*(R1*np.sin(np.radians(theta)/2)+R2*np.cos(np.radians(theta)/2))
        a_=2*(R1*np.cos(np.radians(theta)/2)-R2*np.sin(np.radians(theta)/2))
        
        if edge_mode=='a':#普通こっち
            R1_p=b_*np.sin(np.radians(theta)/2);R2_p=b_*np.cos(np.radians(theta)/2)
        elif edge_mode=='b':
            R1_p=a_*np.sin(np.radians(theta)/2);R2_p=-a_*np.cos(np.radians(theta)/2)

        #映進面によってT配置の分子の向きが違う
        if glide_mode=='b':
            xyz_t= self.make_arr(R1,R2,R3,theta,'t')
        elif glide_mode=='a':
            xyz_t= self.make_arr(R1,R2,R3,theta+180.0,'t')
            
        if torp=='p':
            xyz = self.make_arr(R1_p,R2_p,R3,0.0,'p')
        elif torp=='t':
            xyz = xyz_t
            
        with open(self.path+'for_view\\'+name_gjf,'w') as f:
            f.writelines(xyz[:-1])

############# ファイル処理関係 ##############

    def move_to_4mkdir_step1(self,glide,edge,sep=60):#一フォルダのファイル数
        path_inp=self.path+'step1\\glide='+glide+'\\edge='+edge+'\\'
        #mkdir
        int_folder=0
        file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']
        while len(file_list_inp)!=0:
            
            #フォルダ増やす
            int_folder+=1
            path_inpn=path_inp+'inp'+str(int_folder)
            os.mkdir(path_inpn)
            
            #fileをsepずつ新フォルダに移す
            for file in file_list_inp[:min(len(file_list_inp),sep)]:
                shutil.move(path_inp+file,path_inpn)
            file_list_inp=[file for file in os.listdir(path_inp) if file[-4:]=='.inp']
            
    def move_to_4mkdir_step3(self,glide,edge,sep=30):#一フォルダのファイル数
        #mkdir
        int_folder=0
        file_list_inp=[file for file in os.listdir(self.path+'step3\\glide='+glide+'\\edge='+edge) if file[-4:]=='.inp']
        while len(file_list_inp)!=0:
            #フォルダ増やす
            int_folder+=1
            path_inpn=self.path+'step3\\glide='+glide+'\\edge='+edge+'\\inp'+str(int_folder)
            os.mkdir(path_inpn)
            
            #fileをsepずつ新フォルダに移す
            for file in file_list_inp[:min(len(file_list_inp),sep)]:
                shutil.move(self.path+'step3\\glide='+glide+'\\edge='+edge+'\\'+file,path_inpn)
            file_list_inp=[file for file in os.listdir(self.path+'step3\\glide='+glide+'\\edge='+edge+'\\') if file[-4:]=='.inp']

########   実行ファイル   #######
    def make_exe_step1(self,glide,edge):#実行ファイル作成
        path_inp=self.path+'step1\\glide='+glide+'\\edge='+edge+'\\'
        #mkdir
        file_list_inp=[fold for fold in os.listdir(path_inp) if fold[:3]=='inp']
        N_folder=len(file_list_inp)
        for n_folder in range(1,N_folder+1):
            path_inpn=self.path+'step1\\glide='+glide+'\\edge='+edge+'\\inp'+str(n_folder)
            file_list=[file[:-4] for file in os.listdir(path_inpn) if file[-4:]=='.inp']
            cc_list=[]
            for file_name in file_list:
                line='g16 < '+file_name+'.inp > '+file_name+'.log \n'
                cc_list.append(line)
            with open(path_inpn+'\\exe_6inter.txt','w') as f:
                f.writelines(cc_list)

    def make_exe_step2(self,glide_mode,edge_mode):
        with open(self.path+'step2\\glide='+glide_mode+'\\edge='+edge_mode+'\\exe_6inter.txt','w') as f:
            f.writelines(['g16 < t1.inp > t1.log \n','g16 < t2.inp > t2.log \n','g16 < p.inp > p.log \n'])
    
    def make_exe_step3(self,glide_mode,edge_mode):#実行ファイル作成
        #mkdir
        file_list_inp=[fold for fold in os.listdir(self.path+'step3\\glide='+glide_mode+'\\edge='+edge_mode) if fold[:3]=='inp']
        N_folder=len(file_list_inp)
        for n_folder in range(1,N_folder+1):
            path_inpn=self.path+'step3\\glide='+glide_mode+'\\edge='+edge_mode+'\\inp'+str(n_folder)
            file_list=[file[:-4] for file in os.listdir(path_inpn) if file[-4:]=='.inp']
            cc_list=[]
            for file_name in file_list:
                line='g16 < '+file_name+'.inp > '+file_name+'.log \n'
                cc_list.append(line)
            with open(path_inpn+'\\exe_6inter.txt','w') as f:
                f.writelines(cc_list)
        
#もしminをとらえられていないθがあるなら追加のファイル用意
#glide_mode='a',edge_mode='a',
    def add_step1(self,glide_mode,edge_mode):
        add_list=[]
        
        #計算済みcsv
        df_step1=pd.read_csv(self.path+'step1\\glide={}\\edge={}\\step1.csv'.format(glide_mode,edge_mode))
        min_list=[]
        #min_list=[[5.0,0.6,60],[5.0,1.2,70],,,] minのパラメータセット
        for theta in df_step1['theta'].unique():
            para_list_theta=df_step1[df_step1['theta']==theta].loc[:,['R1','R2','E']].values
            R1,R2,E=para_list_theta[np.argmin(para_list_theta[:,2])]#min時のパラメータを抽出
            
            bool_inc=0
            #minの周りの四つが含まれるか確認
            for R1__,R2__ in para_list_theta[:,:2]:#R1,R2のリスト
                bool_1=(R1+0.1==R1__ and R2==R2__);bool_2=(R1-0.1==R1__ and R2==R2__)
                bool_3=(R1==R1__ and R2+0.1==R2__);bool_4=(R1==R1__ and R2-0.1==R2__)
                bool_inc+=bool_1+bool_2+bool_3+bool_4
            
            #それがまだminじゃないならappend
            if bool_inc==4:
                continue
            elif bool_inc<4:
                min_list.append([R1,R2,theta])

        glide=180.0 if glide_mode=='a' else 0.0
        #はじっこにあるminのヘリンボーンパラメータ入力
        for R1,R2,theta in min_list:
            R1_list=[np.round(R1+d,1) for d in np.linspace(-0.5,0.5,11)]
            R2_list=[np.round(R2+d,1) for d in np.linspace(-1.0,1.0,21)]
            df_calc=df_step1[df_step1['theta']==theta].loc[:,['R1','R2','theta']]
            
            ####R1*R2の直積リスト####
            for R1_,R2_ in itertools.product(R1_list,R2_list):
                #計算済みパラメータとかぶってなければ追加
                if not([R1_,R2_,theta] in df_calc.values.tolist()):
                    add_list.append((R1_,R2_,theta))
        
        
        #make_inp
        for R1,R2,heri in add_list:
            heri_rad=np.radians(heri)
            b_=2*(R1*np.sin(heri_rad/2)+R2*np.cos(heri_rad/2))
            a_=2*(R1*np.cos(heri_rad/2)-R2*np.sin(heri_rad/2))

            #edgeによって平行配置a,bのどちらが近くなるかが変わる。
            if edge_mode=='a':#普通こっち
                R1_p=b_*np.sin(heri_rad/2);R2_p=b_*np.cos(heri_rad/2)
            elif edge_mode=='b':
                R1_p=a_*np.sin(heri_rad/2);R2_p=-a_*np.cos(heri_rad/2) #修正した4/24 edge=bはやり直し。。。

            #配座出力
            xyz_t = self.make_arr(R1,R2,0.0,heri+glide,'t')  
            xyz_p = self.make_arr(R1_p,R2_p,0.0,0.0,'p')
            xyz_list=['$ RunGauss\n']+xyz_t+xyz_p[:-1]#
            with open(self.path+'step1\\glide='+glide_mode+'\\edge='+edge_mode+'\\R1={}_R2={}_theta={}.inp'.format(R1,R2,int(heri)),'w') as f:
                f.writelines(xyz_list)



# STEP1

## ⓪↑の探索範囲をカスタマイズ

## ①インスタンス立ち上げ

In [3]:
maker_1=make_gjf()

## ②inpファイル大量作成

In [4]:
maker_1.make_step1('vdw_b-glide_a-edge_for_inp.csv',glide_mode='b',edge_mode='a')

## ③ファイルを分ける

In [5]:
maker_1.move_to_4mkdir_step1(glide='b',edge='a',sep=60)

## ④実行ファイル

In [6]:
maker_1.make_exe_step1(glide='b',edge='a')

## ⑤追加ファイル→③へ

In [13]:
# maker_1=make_gjf()
# maker_1.add_step1('b','a')#glide,edge

# STEP2

## ①インスタンス

In [3]:
maker_2=make_gjf()

## ②inpファイル大量作成

In [4]:
maker_2.make_step2(5.0,0.8,58,glide_mode='b',edge_mode='a')

## ③実行ファイル

In [5]:
maker_2.make_exe_step2(glide_mode='b',edge_mode='a')

# step3

## ①vdwmin入れる(vdw_contactからとってくる)
## ②vdwmin付近にファイル作成

glide=a

In [9]:
R1_t=5.0;R2_t=0.6;heri=60.0
R3_t=-0.1;R3_p=-0.6;R3_t2=np.round(R3_t-R3_p,1)

##########ここだけいじる
Ra_i0=2.4;Rb_i0=0.2;R3_i0=11.6#vdwmin=2.5,0.3,11.62418694　ここでは0.2Å刻み
##########

maker_3=make_gjf()

In [11]:
for dRa_i0 in np.linspace(-0.2,0.2,3):
    Ra=np.round(Ra_i0+dRa_i0,1)
    for dRb_i0 in np.linspace(-0.2,0.2,3):
        Rb=np.round(Rb_i0+dRb_i0,1)
        for dR3_i0 in np.linspace(-0.6,-0.4,2):
            R3=np.round(R3_i0+dR3_i0,1)
#             maker_3.make_step3(R1_t=R1_t,R2_t=R2_t,heri=heri,
#                                R3_t=R3_t,R3_p=R3_p,
#                                Ra_i0=Ra, Rb_i0=Rb, R3_i0=R3,
#                                glide_mode='a',edge_mode='a',Z=1)
            maker_3.make_step3(R1_t=R1_t,R2_t=R2_t,heri=heri,
                               R3_t=R3_t2,R3_p=-R3_p,
                               Ra_i0=Ra, Rb_i0=-Rb, R3_i0=R3,
                               glide_mode='a',edge_mode='a',Z=2)            

glide=b

In [3]:
R1_t=5.0;R2_t=0.8;heri=58.0
R3_t=-0.3;R3_p=-0.7;R3_t2=np.round(R3_t-R3_p,1)

##########ここだけいじる
Ra_i0=0.0;Rb_i0=-2.6;R3_i0=12.0#11.9454
##########

maker_3=make_gjf()

In [8]:
for dRa_i0 in np.linspace(-0.4,0.4,5):
    Ra=np.round(Ra_i0+dRa_i0,1)
    for dRb_i0 in np.linspace(-0.2,0.2,3):
        Rb=np.round(Rb_i0+dRb_i0,1)
        for dR3_i0 in np.linspace(-0.6,-0.4,2):
            R3=np.round(R3_i0+dR3_i0,1)
            maker_3.make_step3(R1_t=R1_t,R2_t=R2_t,heri=heri,
                                R3_t=R3_t,R3_p=R3_p,
                                Ra_i0=Ra, Rb_i0=Rb, R3_i0=R3,
                                glide_mode='b',edge_mode='a',Z=1)
            maker_3.make_step3(R1_t=R1_t,R2_t=R2_t,heri=heri,
                               R3_t=-R3_t2,R3_p=R3_p,
                               Ra_i0=-Ra, Rb_i0=Rb, R3_i0=R3,
                               glide_mode='b',edge_mode='a',Z=2)            

## ③フォルダ小分け

In [9]:
maker_3.move_to_4mkdir_step3(glide='b',edge='a',sep=15)

## ④実行ファイル作成

In [10]:
maker_3.make_exe_step3(glide_mode='b',edge_mode='a')